<a href="https://www.kaggle.com/code/lovrorabuzin/3drn18-zavrad?scriptVersionId=91692024" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Importi i hiperparametri

In [ ]:
!pip install inplace-abn

In [ ]:
import inplace_abn as iabn

import torch
from torch import nn
from torch.nn import functional
import torch.nn.functional as F

import gc

import sklearn
from sklearn import model_selection

import torchvision
import torchvision.transforms as torch_transforms
import torch.utils.data as data
import torch.optim as optim

from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.utils.data import TensorDataset
from torch.utils.data import Subset

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches

import pickle
import numpy as np
import math
import pandas as pd

import skimage as ski
import skimage.io
import os

import nibabel as nib
from PIL import Image
import imageio

import sys
import json

b_s = 7
learning_rate = 0.001
weight_decay = 0.005
gamma = 0.95
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
root_train_dir = '../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData'

# Učitavanje podataka

In [ ]:
#Ovaj dio koda preuzet je s https://github.com/jcreinhold/niftidataset jer sam imao problema s instalacijom paketa

from typing import Optional, Tuple, Union

class BaseTransform:
    def __repr__(self): return f'{self.__class__.__name__}'

def normalize3d(tensor, mean, std, inplace=False):
    """
    normalize a 3d tensor
    Args:
        tensor (Tensor): Tensor image of size (C, H, W, D) to be normalized.
        mean (sequence): Sequence of means for each channel.
        std (sequence): Sequence of standard deviations for each channel.
    Returns:
        Tensor: Normalized Tensor image.
    """
    if not inplace:
        tensor = tensor.clone()

    mean = torch.as_tensor(mean, dtype=torch.float32, device=tensor.device)
    std = torch.as_tensor(std, dtype=torch.float32, device=tensor.device)
    tensor.sub_(mean[:, None, None, None]).div_(std[:, None, None, None])
    return tensor

class Normalize:
    """
    Implement a normalize function for input two images.
    It computes std and mean for each input Tensor if mean and std equal to None,
     then the function normalizes Tensor using the computed values.
    Args:
        mean: mean of input Tensor. if None passed, mean of each Tensor will be computed and normalization will be performed based on computed mean.
        std: standard deviation of input Tensor. if None passed, std of each Tensor will be computed and normalization will be performed based on computed std.
        tfm_x (bool): transform x or not
        tfm_y (bool): transform y or not
        is_3d (bool): is the Tensor 3d or not. this causes to normalize the Tensor on each channel.
    """

    def __init__(self, mean=None, std=None, tfm_x: bool = True, tfm_y: bool = False,
                 is_3d: bool = False):
        self.mean = mean
        self.std = std
        self.tfm_x = tfm_x
        self.tfm_y = tfm_y
        self.is_3d = is_3d

    def _tfm(self, tensor: torch.Tensor):
        if self.is_3d:
            norm = normalize3d
            mean = torch.as_tensor(self.mean, dtype=torch.float32, device=tensor.device) if not (
                    self.mean is None) else tensor.mean(dim=(1, 2, 3))
            std = torch.as_tensor(self.std, dtype=torch.float32, device=tensor.device) if not (
                    self.std is None) else tensor.std(dim=(1, 2, 3))
            # to prevent division by zero
            std[std == 0.] = 1e-6
        else:
            norm = torchvision.transforms.functional.normalize
            mean = self.mean if not (self.mean is None) else tensor.mean().item()
            std = self.std if not (self.std is None) else tensor.std().item()
            # to prevent division by zero
            if std == 0.:
                std = 1e-6
        return norm(tensor, mean, std)

    def __call__(self, sample: Tuple[torch.Tensor, torch.Tensor]):
        src, tgt = sample
        if self.tfm_x: src = self._tfm(src)
        if self.tfm_y: tgt = self._tfm(tgt)
        return src, tgt

    def __repr__(self):
        s = '{name}(mean={mean}, std={std}, tfm_x={tfm_x}, tfm_y={tfm_y}, is_3d={is_3d})'
        d = dict(self.__dict__)
        return s.format(name=self.__class__.__name__, **d)

class ToTensor(BaseTransform):
    """ Convert images in sample to Tensors """

    def __init__(self, color=False):
        self.color = color

    def __call__(self, sample: Tuple[np.ndarray, np.ndarray]) -> Tuple[torch.Tensor, torch.Tensor]:
        src, tgt = sample
        if isinstance(src, np.ndarray) and isinstance(tgt, np.ndarray):
            return torch.from_numpy(src), torch.from_numpy(tgt)
        if isinstance(src, list): src = np.stack(src)
        if isinstance(tgt, list): src = np.stack(tgt)
        # handle PIL images
        src, tgt = np.asarray(src), np.asarray(tgt)
        if src.ndim == 3 and self.color: src = src.transpose((2, 0, 1)).astype(np.float32)
        if tgt.ndim == 3 and self.color: tgt = tgt.transpose((2, 0, 1)).astype(np.float32)
        if src.ndim == 2: src = src[None, ...]  # add channel dimension
        if tgt.ndim == 2: tgt = tgt[None, ...]
        return torch.from_numpy(src.copy()), torch.from_numpy(tgt.copy())

__all__ = ['NiftiDataset',
           'MultimodalNiftiDataset',
           'MultimodalNifti2p5DDataset',
           'MultimodalImageDataset',
           'train_val_split']

from typing import Callable, List, Optional

import nibabel as nib
import numpy as np
from PIL import Image
from torch.utils.data.dataset import Dataset

from typing import List, Tuple

from glob import glob
import os


def split_filename(filepath: str) -> Tuple[str, str, str]:
    """ split a filepath into the directory, base, and extension """
    path = os.path.dirname(filepath)
    filename = os.path.basename(filepath)
    base, ext = os.path.splitext(filename)
    if ext == '.gz':
        base, ext2 = os.path.splitext(base)
        ext = ext2 + ext
    return path, base, ext


def glob_imgs(path: str, ext='*.nii*') -> List[str]:
    """ grab all `ext` files in a directory and sort them for consistency """
    fns = sorted(glob(os.path.join(path, ext)))
    return fns


class NiftiDataset(Dataset):
    """
    create a dataset class in PyTorch for reading NIfTI files

    Args:
        source_fns (List[str]): list of paths to source images
        target_fns (List[str]): list of paths to target images
        transform (Callable): transform to apply to both source and target images
        preload (bool): load all data when initializing the dataset
    """

    def __init__(self, source_fns: List[str], target_fns: List[str], transform: Optional[Callable] = None,
                 preload: bool = False):
        self.source_fns, self.target_fns = source_fns, target_fns
        self.transform = transform
        self.preload = preload
        if len(self.source_fns) != len(self.target_fns) or len(self.source_fns) == 0:
            raise ValueError(f'Number of source and target images must be equal and non-zero')
        if preload:
            self.imgs = [(nib.load(s).get_data(), nib.load(t).get_data())
                         for s, t in zip(self.source_fns, self.target_fns)]

    @classmethod
    def setup_from_dir(cls, source_dir: str, target_dir: str, transform: Optional[Callable] = None,
                       preload: bool = False):
        source_fns, target_fns = glob_imgs(source_dir), glob_imgs(target_dir)
        return cls(source_fns, target_fns, transform, preload)

    def __len__(self):
        return len(self.source_fns)

    def __getitem__(self, idx: int):
        if not self.preload:
            src_fn, tgt_fn = self.source_fns[idx], self.target_fns[idx]
            sample = (nib.load(src_fn).get_fdata(dtype=np.float32), nib.load(tgt_fn).get_fdata(dtype=np.float32))
        else:
            sample = self.imgs[idx]
        if self.transform is not None:
            sample = self.transform(sample)
        return sample


class MultimodalDataset(Dataset):
    """ base class for Multimodal*Dataset """

    def __init__(self, source_fns: List[List[str]], target_fns: List[List[str]],
                 transform: Optional[Callable] = None,
                 segmentation: bool = False, preload: bool = False, **kwargs):
        self.source_fns, self.target_fns = source_fns, target_fns
        self.transform = transform
        self.segmentation = segmentation
        self.preload = preload
        if any([len(self.source_fns[0]) != len(sfn) for sfn in self.source_fns]) or \
                any([len(self.target_fns[0]) != len(tfn) for tfn in self.target_fns]) or \
                len(self.source_fns[0]) != len(self.target_fns[0]) or \
                len(self.source_fns[0]) == 0:
            raise ValueError(f'Number of source and target images must be equal and non-zero')
        if preload:
            self.imgs = []
            for idx in range(len(self.source_fns[0])):
                src_fns, tgt_fns = [sfns[idx] for sfns in self.source_fns], [tfns[idx] for tfns in self.target_fns]
                self.imgs.append((self.stack([self.get_data(s) for s in src_fns]),
                                  self.stack([self.get_data(t) for t in tgt_fns])))

    @classmethod
    def setup_from_dir(cls, source_dirs: List[str], target_dirs: List[str],
                       transform: Optional[Callable] = None, segmentation: bool = False,
                       preload: bool = False, ext: str = '*.nii*', **kwargs):
        source_fns = [glob_imgs(sd, ext) for sd in source_dirs]
        target_fns = [glob_imgs(td, ext) for td in target_dirs]
        return cls(source_fns, target_fns, transform, segmentation, preload, **kwargs)

    def __len__(self):
        return len(self.source_fns[0])

    def __getitem__(self, idx: int):
        if not self.preload:
            src_fns, tgt_fns = [sfns[idx] for sfns in self.source_fns], [tfns[idx] for tfns in self.target_fns]
            sample = (self.stack([self.get_data(s) for s in src_fns]),
                      self.stack([self.get_data(t) for t in tgt_fns]))
        else:
            sample = self.imgs[idx]
        if self.transform is not None:
            sample = self.transform(sample)
        if self.segmentation:
            sample = (sample[0], sample[1].long()[0])  # for segmentation, loss expects no channel dim
        return sample

    def get_data(self, fn):
        raise NotImplementedError

    def stack(self, imgs):
        raise NotImplementedError


class MultimodalNiftiDataset(MultimodalDataset):
    """
    create a dataset class in PyTorch for reading N types of NIfTI files to M types of output NIfTI files

    ** note that all images must have the same dimensions! **

    Args:
        source_dirs (List[str]): paths to source images
        target_dirs (List[str]): paths to target images
        transform (Callable): transform to apply to both source and target images
    """

    def get_data(self, fn): return nib.load(fn).get_fdata(dtype=np.float32)

    def stack(self, imgs): return np.stack(imgs)


class MultimodalNifti2p5DDataset(MultimodalNiftiDataset):
    """
    create a dataset class in PyTorch for reading N types of NIfTI files to M types of output NIfTI files
    2.5D dataset, so return images stacked in the channel dimension for processing with a 2D CNN

    ** note that all images must have the same dimensions! **

    Args:
        source_dirs (List[str]): paths to source images
        target_dirs (List[str]): paths to target images
        transform (Callable): transform to apply to both source and target images
    """

    def __init__(self, source_dirs: List[str], target_dirs: List[str], transform: Optional[Callable] = None,
                 segmentation: bool = False, preload: bool = False, axis: int = 0):
        self.axis = axis
        super().__init__(source_dirs, target_dirs, transform, segmentation, preload)

    def stack(self, imgs): return np.swapaxes(np.concatenate(imgs, axis=self.axis), 0, self.axis)


class MultimodalImageDataset(MultimodalDataset):
    """
    create a dataset class in PyTorch for reading N types of (no channel) image files to M types of output image files.
    can use whatever PIL can open.

    ** note that all images must have the same dimensions! **

    There is no implementation of ImageDataset because it is sufficient to use normal pytorch image
    dataloaders for that use case

    Args:
        source_dirs (List[str]): paths to source images
        target_dirs (List[str]): paths to target images
        transform (Callable): transform to apply to both source and target images
        color (bool): images are color, ie, 3 channels
    """

    def __init__(self, source_dirs: List[str], target_dirs: List[str], transform: Optional[Callable] = None,
                 segmentation: bool = False, color: bool = False, preload: bool = False):
        self.color = color
        super().__init__(source_dirs, target_dirs, transform, segmentation, preload)

    @classmethod
    def setup_from_dir(cls, source_dirs: List[str], target_dirs: List[str],
                       transform: Optional[Callable] = None,
                       segmentation: bool = False,
                       color: bool = False, preload: bool = False,
                       ext: str = '*.tif*', **kwargs):
        source_fns = [glob_imgs(sd, ext) for sd in source_dirs]
        target_fns = [glob_imgs(td, ext) for td in target_dirs]
        return cls(source_fns, target_fns, transform, segmentation, color, preload)

    def get_data(self, fn):
        data = np.asarray(Image.open(fn), dtype=np.float32)
        if self.color: data = data.transpose((2, 0, 1))
        return data

    def stack(self, imgs):
        data = np.stack(imgs)
        if self.color: data = data.squeeze()
        return data


def train_val_split(source_dir: str, target_dir: str, valid_pct: float = 0.2,
                    transform: Optional[Callable] = None, preload: bool = False):
    """
    create two separate NiftiDatasets in PyTorch for working with NifTi files. If a directory contains source files
    and the other one contains target files and also you dont have a specific directory for validation set,
    this function splits data to two NiftiDatasets randomly with given percentage.

    Args:
        source_dir (str): path to source images.
        target_dir (str): path to target images.
        valid_pct (float): percent of validation set from data.
        transform (Callable): transform to apply to both source and target images.
        preload: load all data when initializing the dataset
    Returns:
        Tuple: (train_dataset, validation_dataset).
    """
    if not (0 < valid_pct < 1):
        raise ValueError(f'valid_pct must be between 0 and 1')
    source_fns, target_fns = glob_imgs(source_dir), glob_imgs(target_dir)
    rand_idx = np.random.permutation(list(range(len(source_fns))))
    cut = int(valid_pct * len(source_fns))
    return (NiftiDataset(source_fns=[source_fns[i] for i in rand_idx[cut:]],
                         target_fns=[target_fns[i] for i in rand_idx[cut:]],
                         transform=transform, preload=preload),
            NiftiDataset(source_fns=[source_fns[i] for i in rand_idx[:cut]],
                         target_fns=[target_fns[i] for i in rand_idx[:cut]],
                         transform=transform, preload=preload))


In [ ]:
tensorize = ToTensor()
normalize = Normalize(is_3d = True)
reorder = torch_transforms.Lambda(lambda t: (t[0].permute(0,3,1,2), t[1].permute(0,3,1,2)))

tr = torch_transforms.Compose([tensorize, normalize, reorder])

name_mapping = pd.read_csv(root_train_dir + '/name_mapping.csv')

name_mapping.rename({'BraTS_2020_subject_ID': 'ID'}, axis=1, inplace=True)

survival_info = pd.read_csv(root_train_dir + '/survival_info.csv')

survival_info.rename({'Brats20ID': 'ID'}, axis=1, inplace=True)

patient_info = survival_info.merge(name_mapping, on="ID", how="right")

modalities = ['_flair.nii', '_t1.nii', '_t1ce.nii', '_t2.nii']

train_images = []
valid_images = []
test_images = []

HGG_names = list(patient_info[patient_info['Grade'] == 'HGG'].ID)
LGG_names = list(patient_info[patient_info['Grade'] == 'LGG'].ID)

for i in range(len(HGG_names)):
    image_path = root_train_dir+'/'+HGG_names[i]+'/'+HGG_names[i]
    if i % 7 == 5:
        valid_images.append(image_path)
    elif i % 7 == 4:
        test_images.append(image_path)
    else:
        train_images.append(image_path)

for i in range(len(LGG_names)):
    image_path = root_train_dir+'/'+LGG_names[i]+'/'+LGG_names[i]
    if i % 7 == 5:
        valid_images.append(image_path)
    elif i % 7 == 4:
        test_images.append(image_path)
    else:
        train_images.append(image_path)

train_sources = []
valid_sources = []
test_sources = []

modality_names=[]

for modality in modalities:
    modality_names.append(modality[1:-4])
    train_sources.append([image + modality for image in train_images])
    valid_sources.append([image + modality for image in valid_images])
    test_sources.append([image + modality for image in test_images])

train_targets = []
valid_targets = []
test_targets = []

for modality in modalities:
    train_targets.append([image + '_seg.nii' for image in train_images])
    valid_targets.append([image + '_seg.nii' for image in valid_images])
    test_targets.append([image + '_seg.nii' for image in test_images])
    
    real_name = '../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_355/W39_1998.09.19_Segm.nii'
    fake_name = '../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_355/BraTS20_Training_355_seg.nii'
    valid_targets = [[real_name if (x == fake_name) else x for x in valid_targets_in] for valid_targets_in in valid_targets]

trainset = MultimodalNiftiDataset(train_sources, train_targets, tr, segmentation = True)
validset = MultimodalNiftiDataset(valid_sources, valid_targets, tr, segmentation = True)
testset = MultimodalNiftiDataset(test_sources, test_targets, tr, segmentation = True)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=b_s, shuffle=True, num_workers=0)
validloader = torch.utils.data.DataLoader(validset, batch_size=b_s,shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(testset, batch_size=b_s, shuffle=False, num_workers=0)

# Neuronska mreža (3D ResNet18+SPP)

In [ ]:
upsample = lambda x, size: F.interpolate(x, size, mode='trilinear', align_corners=False)

class ResidualBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, channels, stride=1, identity_downsample=None):
        super(ResidualBlock, self).__init__()
        self.bn1 = iabn.ABN(in_channels)
        self.conv1 = nn.Conv3d(in_channels, channels, kernel_size = 3, stride = stride, padding = 1, bias = False)
        self.bn2 = iabn.ABN(channels)
        self.conv2 = nn.Conv3d(channels, channels, kernel_size = 3, padding = 1, bias = False)
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):        
        x = self.bn1(x)
        residual = x.clone()
        x = self.conv1(x)
        x = self.bn2(x)
        x = self.conv2(x)
        if self.identity_downsample is not None:
            residual = self.identity_downsample(residual)
        x += residual
        return x, x

class BNReLUConv(nn.Sequential):
    def __init__(self, in_channels, out_channels, k=3):
        super(BNReLUConv, self).__init__()
        self.add_module('BN', iabn.ABN(in_channels))
        padding = k // 2
        self.add_module('Conv', nn.Conv3d(in_channels, out_channels, k, padding=padding, bias=False))
        
class Up(nn.Module):
    def __init__(self, in_channels, skip_channels, out_channels, k=3):
        super(Up, self).__init__()
        self.bottleneck = BNReLUConv(skip_channels, in_channels, k = 1)
        self.blend = BNReLUConv(in_channels, out_channels, k = k)
        self.upsample = lambda x, size: F.interpolate(x, size, mode='trilinear', align_corners=False)
    
    def forward(self, x, skip):
        skip = self.bottleneck.forward(skip)
        skip_size = skip.size()[2:5]
        x = self.upsample(x, skip_size)
        x = x + skip
        x = self.blend.forward(x)
        return x

class SPP(nn.Module):
    def __init__(self, in_channels, num_levels, bt_size, level_size, out_size, grids):
        super(SPP, self).__init__()
        self.upsample = lambda x, size: F.interpolate(x, size, mode='trilinear', align_corners=False)
        self.spp = nn.Sequential()
        self.spp.add_module('spp_bn', BNReLUConv(in_channels, bt_size, k=1))
        self.grids = grids
        num_features = bt_size
        final_size = num_features
        for i in range(num_levels):
            final_size += level_size
            self.spp.add_module('spp' + str(i), BNReLUConv(num_features, level_size, k=1))
        self.spp.add_module('spp_fuse', BNReLUConv(final_size, out_size, k=1))
        
    def forward(self, x):
        levels = []
        target_size = x.size()[2:5]
        ar = target_size[0]/target_size[1]
        x = self.spp[0].forward(x)
        levels.append(x)
        num = len(self.spp) - 1
        for i in range(1, num):
            grid_size = (max(1, round(ar * self.grids[i - 1])), self.grids[i-1], self.grids[i-1]) # 
            x_pooled = F.adaptive_avg_pool3d(x, grid_size) #
            level = self.spp[i].forward(x_pooled)
            level = self.upsample(level, target_size)
            levels.append(level)
        x = torch.cat(levels, 1)
        x = self.spp[-1].forward(x)
        return x

class ResNet(nn.Module):
    def __init__(self, residual_block, layers, image_channels=4):
        super(ResNet, self).__init__()
        self.inplanes = 32
        self.num_features = 128
        self.bn1 = iabn.ABN(4)
        self.conv1 = nn.Conv3d(4, 32, kernel_size = 7, stride = 2, padding = 3, bias = False)
        self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(residual_block, 32, layers[0])
        self.layer2 = self._make_layer(residual_block, 64, layers[1], stride=2)
        self.layer3 = self._make_layer(residual_block, 128, layers[2], stride=2)
        self.layer4 = self._make_layer(residual_block, 256, layers[3], stride=2)
        num_levels = 3
        self.spp_size = 128
        bt_size = self.spp_size
        level_size = self.spp_size // num_levels
        self.spp = SPP(self.inplanes, num_levels, bt_size=bt_size, level_size=level_size, out_size=128, grids = (4,2,1))
        self.upblend = nn.Conv3d(128, 4, kernel_size = 3, stride = 1, padding = 1)
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
    
    def forward_resblock(self, x, layers):
        skip = None
        for l in layers:
            x = l(x)
            if isinstance(x, tuple):
                x, skip = x
        return x, skip

    def forward_down(self, image):
        x = self.bn1(image)
        x = self.conv1(x)
        x = self.maxpool(x)
        features = []
        x, skip = self.forward_resblock(x, self.layer1)
        x, skip = self.forward_resblock(x, self.layer2)
        x, skip = self.forward_resblock(x, self.layer3)
        x, skip = self.forward_resblock(x, self.layer4)
        x = self.spp.forward(skip)
        image = self.upblend(x)
        return image

    def forward(self, image):
        return self.forward_down(image)
    
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            layers = [iabn.ABN(self.inplanes)]
            layers += [nn.Conv3d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False)]
            downsample = nn.Sequential(*layers)
        layers = [block(self.inplanes, planes, stride, downsample)]
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers += [block(self.inplanes, planes)]
        return nn.Sequential(*layers)

class SemSegModel(nn.Module):
    def __init__(self, backbone, num_classes):
        super(SemSegModel, self).__init__()
        self.backbone = backbone
        self.num_classes = num_classes
        self.logits = BNReLUConv(backbone.num_features, self.num_classes)
        self.multiscale_factors=(.5, .75, 1.5, 2.)
        
    def forward(self, image):
        image_size = image.size()[2:5]
        features = self.backbone(image)
        logits = upsample(features, image_size)
        return logits

# Pomoćne funkcije

In [ ]:
def split_segments(target, batch_size):
    layers = []
    for i in range(batch_size):
        gt_bg = torch.zeros(target[i].shape)
        gt_pe = torch.zeros(target[i].shape)
        gt_ec = torch.zeros(target[i].shape)
        gt_nc = torch.zeros(target[i].shape)
        gt_bg[target[i] == 0] = 1
        gt_pe[target[i] == 2] = 1
        gt_ec[target[i] == 4] = 1
        gt_nc[target[i] == 1] = 1
        layers.append(torch.stack([gt_pe,gt_nc,gt_ec,gt_bg]))
    return torch.stack(layers)

def un_one_hot(target):
    return(target.argmax(1))

def dice_score(prediction, ground_truth, smooth = 1.0):
    n = prediction.shape[0]
    score_acc = 0
    for i in range(n):
        prediction_inner = prediction[i].contiguous().view(-1)
        ground_truth_inner = ground_truth[i].contiguous().view(-1)
    
        intersection = (prediction_inner*ground_truth_inner).sum()
        score = (2*intersection+smooth)/(prediction_inner.sum()+ground_truth_inner.sum()+smooth)
        score_acc+=score
    return score/n

def evaluate(model, loader, loss_fn):
    global device
    
    loss_avg = 0.0
    dice_scores = []
    
    with torch.no_grad():
        model.eval()
        for images, ground_truths in loader:
            images = images.to(device)
            ground_truths = ground_truths.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, un_one_hot(split_segments(ground_truths, ground_truths.shape[0])).to(device=device))
            loss_avg += loss.item()
            outputs = nn.LogSoftmax(1)(outputs)
            outputs = outputs.argmax(1)
            ground_truths = un_one_hot(split_segments(ground_truths, ground_truths.shape[0]).to(device=device))

            outputs_WT = outputs.clone()
            outputs_WT[outputs_WT == 0] = 1
            outputs_WT[outputs_WT == 1] = 1
            outputs_WT[outputs_WT == 2] = 1
            outputs_WT[outputs_WT == 3] = 0

            outputs_TC = outputs.clone()
            outputs_TC[outputs_TC == 0] = 0
            outputs_TC[outputs_TC == 1] = 1
            outputs_TC[outputs_TC == 2] = 1
            outputs_TC[outputs_TC == 3] = 0

            outputs_ET = outputs.clone()
            outputs_ET[outputs_ET == 0] = 0
            outputs_ET[outputs_ET == 1] = 0
            outputs_ET[outputs_ET == 2] = 1
            outputs_ET[outputs_ET == 3] = 0
            
            ground_truths_WT = ground_truths.clone()
            ground_truths_WT[ground_truths_WT == 0] = 1
            ground_truths_WT[ground_truths_WT == 1] = 1
            ground_truths_WT[ground_truths_WT == 2] = 1
            ground_truths_WT[ground_truths_WT == 3] = 0

            ground_truths_TC = ground_truths.clone()
            ground_truths_TC[ground_truths_TC == 0] = 0
            ground_truths_TC[ground_truths_TC == 1] = 1
            ground_truths_TC[ground_truths_TC == 2] = 1
            ground_truths_TC[ground_truths_TC == 3] = 0

            ground_truths_ET = ground_truths.clone()
            ground_truths_ET[ground_truths_ET == 0] = 0
            ground_truths_ET[ground_truths_ET == 1] = 0
            ground_truths_ET[ground_truths_ET == 2] = 1
            ground_truths_ET[ground_truths_ET == 3] = 0
            dice_scores.append(torch.tensor([dice_score(outputs_WT, ground_truths_WT),
                                dice_score(outputs_TC, ground_truths_TC),
                                dice_score(outputs_ET, ground_truths_ET)]))
    del images, ground_truths, outputs, outputs_WT, outputs_TC, outputs_ET, ground_truths_WT, ground_truths_TC, ground_truths_ET
    gc.collect()
    loss_avg /= len(loader)
    model.train()
    dice_scores = torch.stack(dice_scores)
    return loss_avg, dice_scores.mean(0)

def output_metrics(loss_avg, dice_scores, phase):
    scores = dice_scores
    print(phase)
    print("Average loss: {}".format(loss_avg))
    print("Dice scores - peritumoral edema:{}, necrotic core:{}, enhancing core:{}".format(scores[0], scores[1], scores[2]))

# Iscrtavanje grafa gubitka

In [ ]:
def plot_progress(data):
    valid_loss = data['valid loss']
    train_loss = data['train loss']

    fig, ax = plt.subplots(figsize=(16,8))
    linewidth = 2
    legend_size = 10
    train_color = 'm'
    val_color = 'c'

    ax.set_title('Loss')
    ax.plot(train_loss, marker='o', color=train_color,
           linewidth=linewidth, linestyle='-', label='train')
    ax.plot(valid_loss, marker='o', color=val_color,
           linewidth=linewidth, linestyle='-', label='validation')
    ax.legend(loc='upper right', fontsize=legend_size)

    save_path = os.path.join('./', 'loss.png')
    print('Plotting in: ', save_path)
    plt.savefig(save_path)
    return

# Treniranje i evaluacija modela

In [ ]:
def trainNetwork():
    global device
    plot_data = {}
    plot_data["train loss"] = []
    plot_data["valid loss"] = []
    plot_data["valid dice WT"] = []
    plot_data["valid dice NC"] = []
    plot_data["valid dice ET"] = []
    plot_data["lr"] = []
    
    SAVE_DIR = "./network.pt"
    
    print("device:", device)

    net = SemSegModel(ResNet(ResidualBlock, [2,2,2,2]), 4).to(device = device)
    
    lossFunc = nn.CrossEntropyLoss(weight = torch.tensor([2,5,3,1]).float().to(device=device))
    
    optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay = weight_decay)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)
    
    for e in range(num_epochs):
        accLoss = 0.
        net.train()
        for i, data in enumerate(trainloader):
            inputs, ground_truths = data
            inputs = inputs.to(device = device)
            ground_truths = ground_truths.to(device=device)
            
            outputs = net(inputs)
            ground_truths = un_one_hot(split_segments(ground_truths, ground_truths.shape[0])).to(device=device)
            loss = lossFunc(outputs, ground_truths)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            accLoss += loss.item()
            
            if i % 10 == 0:
                print("Epoch: %d, Iteration: %5d, Loss: %.3f" % ((e + 1), (i+1), (accLoss / (i + 1))))
            del loss, outputs, inputs, ground_truths
            gc.collect()
            
        plot_data["train loss"].append(accLoss/(i+1))
        val_loss, val_dice = evaluate(net, validloader, lossFunc)
        output_metrics(val_loss, val_dice, "Validation")
        plot_data["valid loss"].append(val_loss)
        plot_data["valid dice WT"].append(val_dice[0])
        plot_data["valid dice NC"].append(val_dice[0])
        plot_data["valid dice ET"].append(val_dice[0])
        plot_data["lr"].append(scheduler.get_last_lr())
        
        gc.collect()
        
        scheduler.step()
    
    test_loss, test_dice = evaluate(net, testloader, lossFunc)
    output_metrics(test_loss, test_dice, "Test")
    
    torch.save(net.state_dict(), SAVE_DIR)
    
    plot_data["test loss"] = test_loss
    plot_data["test dice"] = test_dice
    
    try:
        with open("./epoch_data.txt", 'w') as f:
            f.write(repr(plot_data))
        plot_progress(plot_data)
    except:
        print("Oops")
    
    return plot_data
    
epoch_data = trainNetwork()